# TLC data downloading and processing

#### Imports

In [1]:
import os
import time
from downloader import download_file
from IPython.core.display import clear_output

import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

#### Set data directory

In [2]:
puidata = os.getenv("PUIDATA")
if puidata is None:
    os.environ["PUIDATA"] = "{}/data/PUIdata".format(os.getenv("HOME"))
    puidata = os.getenv("PUIDATA")
    print("Warning: PUIDATA environmental variable not found and set by code, please review!")
print("PUIDATA: {}".format(puidata))

PUIDATA: /nfshome/pmb434/PUIdata


## Define taxi zones for analysis

#### Download TLC taxi zones shapefile

In [3]:
#Download shapefile
taxi_zones_files = download_file(url='https://s3.amazonaws.com/nyc-tlc/misc/taxi_zones.zip',
                                 filename='taxi_zones.zip',
                                 destination=puidata)

#Read to GeoDataFrame
taxi_zones_shp = gpd.read_file(taxi_zones_files[2])
taxi_zones_shp.head()

File already downloaded.
File already extracted.
taxi_zones.zip contents in place, you can continue.


,OBJECTID,Shape_Leng,Shape_Area,zone,LocationID,borough,geometry
0,1,0.116357,0.000782,Newark Airport,1,EWR,"POLYGON ((933100.9183527103 192536.0856972019,..."
1,2,0.433470,0.004866,Jamaica Bay,2,Queens,"(POLYGON ((1033269.243591294 172126.0078125, 1..."
2,3,0.084341,0.000314,Allerton/Pelham Gardens,3,Bronx,"POLYGON ((1026308.769506663 256767.6975403726,..."
3,4,0.043567,0.000112,Alphabet City,4,Manhattan,"POLYGON ((992073.4667968601 203714.0759887695,..."
4,5,0.092146,0.000498,Arden Heights,5,Staten Island,"POLYGON ((935843.3104932606 144283.335850656, ..."


#### Select taxi zones for analysis

In [4]:
#Selected taxi zones
taxi_zones = [140, 141, 262, 263, 75, 236, 237]
taxi_zones_shp = taxi_zones_shp[taxi_zones_shp.LocationID.isin(taxi_zones)]

#Get bounds to reduce trips dataframe for spatial join
taxi_zones_shp.to_crs(epsg=4326, inplace=True)
zone_bounds = taxi_zones_shp.total_bounds
taxi_zones_shp.to_crs(epsg=2263, inplace=True)
zone_bounds

array([-73.97301487,  40.75827092, -73.935053  ,  40.79812922])

## Download and process each month's trips dataset

In [10]:
#Needed for months with wrong formatting (2 empty columns at the end)
columns = ['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag',
       'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra',
       'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
       'total_amount','error','error2']

#Empty Dataframe to fill with data
all_months = pd.DataFrame(columns=['PULocationID', 'date', 'passenger_count'])

#Counter for display
i=0

#Nested loop: year + month
for y in range(2014, 2019):
    
    for m in range(1,13):
        
        #Get only the last four years, ending in Jul 2018 (latest available dataset)
        if (y == 2018 and m > 6) or (y == 2014 and m < 7):
            continue
         
        i = i+1
        
        #Set filename and url
        month = str(m) if len(str(m)) == 2 else '0' + str(m)
        filename = 'yellow_tripdata_' + str(y) + '-' + month + '.csv'
        url = "https://s3.amazonaws.com/nyc-tlc/trip+data/" + filename
        
        print('File {}: {}'.format(i, filename))
        
        #Check if we already processed this month
        if os.path.isfile(puidata + '/yellowtaxi_partial_{}-{}.csv'.format(y,month)):
            
            print('Already processed, loading partial file')
            
            all_months = all_months.append(
                pd.read_csv(puidata + '/yellowtaxi_partial_{}-{}.csv'.format(y,month), parse_dates=[1]))
        
        else:
            
            #Download month dataset
            path = download_file(url=url, filename=filename, destination=puidata)

            print('Reading')

            #Months with wrong column formatting
            if y == 2016 and m > 6:
                month_df = pd.read_csv(path, parse_dates=[1], infer_datetime_format=True,
                               index_col=False, names=columns, skiprows=[0])
            else:
                month_df = pd.read_csv(path, parse_dates=[1], infer_datetime_format=True, skipinitialspace=True)
            
            #Before Jul 2016 data has latlong for each trips pickup and drop off location
            #Convert to taxi zones - We are only using pickup location
            if (y < 2016) or (y == 2016 and m < 7):
                
                month_select = month_df[month_df.pickup_longitude.between(zone_bounds[0], zone_bounds[2]) & \
                                        month_df.pickup_latitude.between(zone_bounds[1], zone_bounds[3])]


                print('Converting to GeoDataFrame')

                geometry = [Point(xy) for xy in zip(month_select.pickup_longitude, month_select.pickup_latitude)]
                crs = {'init': 'epsg:4326'}
                month_select = gpd.GeoDataFrame(month_select, crs=crs, geometry=geometry)
                month_select.to_crs(epsg=2263, inplace=True)

                print('Spatial join')

                month_select = gpd.sjoin(month_select, taxi_zones_shp)
                month_select.rename(columns={'LocationID':'PULocationID'}, inplace=True)

            else:
                
                month_select = month_df[month_df.PULocationID.isin(taxi_zones)]
                
            print('Grouping')

            #Column changed name at some point
            if 'tpep_pickup_datetime' in month_select.columns:
                month_select['date'] = month_select.tpep_pickup_datetime.dt.date
            else:
                month_select['date'] = month_select.pickup_datetime.dt.date

            #Group by date and pickup taxi zone ID -> Total passengers per day
            month_grouped =  month_select.groupby(['PULocationID','date'], as_index=False)[['passenger_count']].sum()
            
            #Remove dates out of range (happens in some months)
            month_grouped = month_grouped[(pd.datetime(y, m, 1).date() <= month_grouped.date) & \
                                          (month_grouped.date < (pd.datetime(y, m, 1) + pd.DateOffset(months=1)).date())]
            
            print('Saving partial file')
            
            #Save partial file for convenience
            month_grouped.to_csv(puidata + '/yellowtaxi_partial_{}-{}.csv'.format(y,month), index=False)

            #Append processed data to DF with all months
            all_months = all_months.append(month_grouped)
        
        clear_output(wait=True)
        time.sleep(1)

print('Done!')

#Save all months
all_months.to_csv(puidata + '/yellowtaxi_all.csv', index=False)
all_months.head()

Done!


,PULocationID,date,passenger_count
0,75.0,2014-07-01,3607.0
1,75.0,2014-07-02,4240.0
2,75.0,2014-07-03,4342.0
3,75.0,2014-07-04,3345.0
4,75.0,2014-07-05,3144.0


In [11]:
all_months.PULocationID.value_counts()

263.0    1461
262.0    1461
237.0    1461
236.0    1461
141.0    1461
140.0    1461
75.0     1461
Name: PULocationID, dtype: int64